In [1]:
import tensorflow as tf
import pickle
from tensorflow.keras import  preprocessing, utils
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np 
import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# tf.logging.set_verbosity(tf.logging.ERROR)
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
category_names = ['sentimental' ,'afraid', 'proud', 'faithful', 'joyful', 'angry', 'annoyed', 'disgusted', 'anticipating','sad', 'neutral', 'surprised']
index = {w:i for i, w in enumerate(category_names)}
index

{'sentimental': 0,
 'afraid': 1,
 'proud': 2,
 'faithful': 3,
 'joyful': 4,
 'angry': 5,
 'annoyed': 6,
 'disgusted': 7,
 'anticipating': 8,
 'sad': 9,
 'neutral': 10,
 'surprised': 11}

In [3]:
def token(questions , answers, w):
        oov = [0,0,0,0,0,0,0,-1,-1,-1,0,0]
        tokenizer = preprocessing.text.Tokenizer(oov_token=1)
        # tokenizer = preprocessing.text.Tokenizer(oov_token=1)
        # tokenizer = get_tokenizer(w)
        tokenizer.fit_on_texts( questions + answers )
        VOCAB_SIZE = len( tokenizer.word_index ) + 1 + oov[index[w]]
        # print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))
        tokenizer

        vocab = []
        for word in tokenizer.word_index:
            vocab.append(word)
        
        maxlen_questions = int(np.ceil(np.mean([len(q) for q in questions])))
        maxlen_answers = int(np.ceil(np.mean([len(a) for a in answers])))
        
        # print(maxlen_questions , maxlen_answers)
        tokenized_questions = tokenizer.texts_to_sequences( questions  )
        padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post' ,truncating= 'post')
        encoder_input_data = np.array(padded_questions)


        tokenized_answers = tokenizer.texts_to_sequences( answers  )
        padded_answers = np.array(preprocessing.sequence.pad_sequences( tokenized_answers , maxlen= maxlen_answers , padding='post',truncating= 'post' ))
        decoder_input_data = np.array( padded_answers )


        for i in range(len(tokenized_answers)) :
            tokenized_answers[i] = tokenized_answers[i][1:]
        padded_answers1 = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post',truncating= 'post'  )
        # onehot_answers = utils.to_categorical( padded_answers1 , num_classes= VOCAB_SIZE ,dtype = 'float32' )
        decoder_output_data = np.array( padded_answers1 )
        return VOCAB_SIZE ,maxlen_questions, maxlen_answers,encoder_input_data ,decoder_input_data ,decoder_output_data ,tokenizer

In [4]:
def MA(words, message):    
    
    with open ('processed_dataset/' + words + '.pickle' , 'rb+') as f: 
        questions ,answers =  pickle.load(f)
    VOCAB_SIZE, maxlen_questions, maxlen_answers, encoder_input_data, decoder_input_data, decoder_output_data ,tokenizer= token(questions , answers, words)
    # print(VOCAB_SIZE, maxlen_questions, maxlen_answers, encoder_input_data.shape, decoder_input_data.shape, decoder_output_data.shape)
    def custom_loss_function(y_true, y_pred):
        y1 = tf.numpy_function(utils.to_categorical , [y_true ,VOCAB_SIZE] , tf.float32) 
        cce = tf.keras.losses.CategoricalCrossentropy()
    
        loss = cce(y1, y_pred)
    # del y1
        return loss
    # print("alpha")
    encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
    encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
    encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
    encoder_states = [ state_h , state_c ]

    decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
    decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
    decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
    decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
    output = decoder_dense ( decoder_outputs )
    model3 = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
    model3.compile(optimizer=tf.keras.optimizers.RMSprop(), loss=custom_loss_function , run_eagerly= False )
    
    model3.load_weights('Models/' + words+'.h5')
    
    def make_inference_models():
        
        encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
        decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
        decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
        
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        
        decoder_outputs, state_h, state_c = decoder_lstm(
            decoder_embedding , initial_state=decoder_states_inputs)
        
        decoder_states = [state_h, state_c]

        decoder_outputs = decoder_dense(decoder_outputs)
        
        decoder_model = tf.keras.models.Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        
        return encoder_model , decoder_model
    
    def str_to_tokens( sentence : str ):

        words = sentence.lower().split()
        tokens_list = list()
    
        for word in words:
            tokens_list.append( tokenizer.word_index[ word ] ) 
        return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')
    enc_model , dec_model = make_inference_models()

    # for _ in range(10):
    # print("beta")
    ads = str_to_tokens( message)
    # print(ads)
    states_values = enc_model.predict(ads)
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        if sampled_word_index == 0 :
            sampled_word_index =8
        # sampled_word = None
        # print(sampled_word_index)
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > 10:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    # print(decoded_translation)
    questions.clear()
    answers.clear()
    return decoded_translation 
    

In [5]:
def sentimental_classifer(message):
   f = open("Models/tfdf.pickle", "rb+")
   tfidf = pickle.load(f)
   f.close()

   message1 = []
   message1.append(message)
   x = tfidf.transform(message1).astype('float16')
   model_sentiment_dnn = tf.keras.models.load_model('Models/model_dnn.h5')
   onehot =  model_sentiment_dnn.predict(x)

   idx = np.argmax(onehot, axis=1)[0]
   unique_sentiments = ['sentimental', 'afraid', 'proud', 'faithful', 'joyful', 'angry', 'sad', 'annoyed', 'disgusted', 'anticipating', 'neutral', 'surprised']
   sentiment = unique_sentiments[idx]
   print("sentiment: ", sentiment)
   result = MA(sentiment,message)
   return result
    


In [7]:
while True:
    message = input()
    print(sentimental_classifer(message))

sentiment:  surprised
 i am so excited end
sentiment:  sad
 it see thank it thanks get win her a i it
sentiment:  joyful
 i am so happy to have a good time end
sentiment:  faithful
 i am so glad you are going to do great end
sentiment:  faithful
 i am so glad you are going to do great end


KeyboardInterrupt: Interrupted by user